In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (6800 Fisher Island Dr #6834, Miami Beach, FL ...
2     (800 South Pointe Dr APT 604, Miami Beach, FL ...
3     (18975 Collins Ave #04, Sunny Isles Beach, FL ...
4     (9701 Collins Ave #2302s, Miami Beach, FL 3315...
5     (1455 Ocean Dr, Miami Beach, FL 33139, USA, (2...
6     (18975 Collins Ave #5100, Sunny Isles Beach, F...
7     (18501 Collins Ave #2702, Sunny Isles Beach, F...
8     (18555 Collins Ave Apt 2005, Sunny Isles Beach...
9     (5500 Island Estates Dr #1505, Aventura, FL 33...
10    (10295 Collins Ave UNIT 603, Bal Harbour, FL 3...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Palazzo Della Luna\r6800 Fisher Island Dr 6834...,1/28/2022,4/11/2022,4,"$15,000,000.00",4757,"$3,153.25",Sandra Fiorenza,Archie Drury,Palazzo Della Luna 6800 Fisher Island Dr 6834 ...,Palazzo Della Luna,6800 Fisher Island Dr 6834 Miami Beach,"(6800 Fisher Island Dr #6834, Miami Beach, FL ...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
2,Apogee Condo\r800 S Pointe Dr 604\rMiami Beach,4/2/2021,4/12/2022,341,"$14,500,000.00",6595,"$3,490.61",Dora Puig,Barbie Castro,Apogee Condo 800 S Pointe Dr 604 Miami Beach,Apogee Condo,800 S Pointe Dr 604 Miami Beach,"(800 South Pointe Dr APT 604, Miami Beach, FL ...","(25.7677245, -80.13634490000001, 0.0)",25.767724,-80.136345,0.0
3,Residences by Armani/Casa\r18975 Collins Ave P...,4/6/2022,4/11/2022,1,"$12,400,000.00",4821,"$2,572.08",Daniel Tzinker,Rachel Wertheimer,Residences by Armani/Casa 18975 Collins Ave PH...,Residences by Armani/Casa,18975 Collins Ave PH-04 Sunny Isles Beach,"(18975 Collins Ave #04, Sunny Isles Beach, FL ...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0
4,St Regis\r9701 Collins Ave 2302S\rBal Harbour,2/28/2022,4/13/2022,21,"$8,400,000.00",4141,"$2,681.14",Isaac Lustgarten,Menachem Fellig,St Regis 9701 Collins Ave 2302S Bal Harbour,St Regis,9701 Collins Ave 2302S Bal Harbour,"(9701 Collins Ave #2302s, Miami Beach, FL 3315...","(25.8884913, -80.1228812, 0.0)",25.888491,-80.122881,0.0
5,Il Villaggio\r1455 Ocean Dr 1603/4\rMiami Beach,6/25/2021,4/11/2022,259,"$6,900,000.00",3212,"$2,148.19",Eloy Carmenate,Maritza Cuellar,Il Villaggio 1455 Ocean Dr 1603/4 Miami Beach,Il Villaggio,1455 Ocean Dr 1603/4 Miami Beach,"(1455 Ocean Dr, Miami Beach, FL 33139, USA, (2...","(25.7873417, -80.129359, 0.0)",25.787342,-80.129359,0.0
6,Residences by Armani/Casa\r18975 Collins Ave 5...,5/13/2021,4/12/2022,286,"$6,150,000.00",3471,"$1,771.82",Jonathan Mann,Helena Fordin,Residences by Armani/Casa 18975 Collins Ave 51...,Residences by Armani/Casa,18975 Collins Ave 5100 Sunny Isles Beach,"(18975 Collins Ave #5100, Sunny Isles Beach, F...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0
7,Turnberry Ocean Club\r18501 Collins Ave 2702\r...,5/28/2021,4/14/2022,299,"$5,100,000.00",3760,"$1,639.87",Jaime Suarez,Lindsey Brown,Turnberry Ocean Club 18501 Collins Ave 2702 Su...,Turnberry Ocean Club,18501 Collins Ave 2702 Sunny Isles Beach,"(18501 Collins Ave #2702, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
8,Porsche Design Tower\r18555 Collins Ave 2005\r...,11/22/2021,4/11/2022,101,"$4,600,000.00",3171,"$1,450.65",Patricia Bastidas,Melissa Barragan,Porsche Design Tower 18555 Collins Ave 2005 Su...,Porsche Design Tower,18555 Collins Ave 2005 Sunny Isles Beach,"(18555 Collins Ave Apt 2005, Sunny Isles Beach...","(25.9481501, -80.11994659999999, 0.0)",25.948150,-80.119947,0.0
9,Prive\r5500 Island Estates Dr 1505\rAventura,1/12/2022,4/12/2022,78,"$4,365,000.00",3984,"$1,095.63",Michael Neumann,Michael Neumann,Prive 5500 Island Estates Dr 1505 Aventura,Prive,5500 Island Estates Dr 1505 Aventura,"(5500 Island Estates Dr #1505, Aventura, FL 33...","(25.946531, -80.130636, 0.0)",25.946531,-80.130636,0.0
10,One Bal Harbour\r10295 Collins Ave 603\rBal Ha...,5/28/2021,4/12/2022,238,"$4,000,000.00",3105,"$1,288.24",Gisela Cacciamani,Lourdes Lorenzo-Luaces,One Bal Harbour 10295 Collins Ave 603 Bal Harbour,One Bal Harbour,10295 Collins Ave 603 Bal Harbour,"(10295 Collins Ave UNIT 603, Bal Harbour, FL 3...","(25.8991765, -80.12308530000001, 0.0)",25.899176,-80.123085,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [14]:
df.at[5,'Agent']=('Eloy Carmenate & Mick Duchon')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 10th - April 16th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [16]:
m.save('index.html')